In [35]:
import numpy as np
# CONSTANTS - DO NOT CHANGE
G = 6.67E-11 # Gravitational Constant 
g_0 = 9.8 # Acceleration due to gravity on earth

In [46]:
# Parameters
M_M = 5.972E24 # Mass of orbited body (kg)
R_M = 6371 # Radius of orbited body (km)
w_M = 7.11E-5 # Equatorial angular velocity (rad/s)

mpay = 10000 # payload mass (kg)

Isp_1 = 310 # First Stage Isp (s)
Isp_2 = 340 # Second Stage Isp (s)
fInert1 = .3 # First Stage inert mass fraction 
fInert2 = .25 # Second Stage inert Mass fraction

site_lattitude = 40 # Launch Site lattitude (degrees)
target_inclination = 60 # Target orbital inclination (degrees)
target_altitude = 100 # Target orbital altittude (km)

twr_1 = 1.4 # Thrust to weight ratio for stage one, recommended > 1.4
twr_2 = 1.3 # Thrust to weight ratio for stage two, recommended > 1.4

In [47]:
g_M = G*M_M/(R_M*1000)**2 # Acceleration due to gravity for orbited body (m/s^2)

lat_rad = np.radians(site_lattitude)
inc_rad = np.radians(target_inclination)
            
azimuth = np.degrees(np.arcsin(np.cos(inc_rad)/np.cos(lat_rad))) # degrees
        

dV_earth_help = w_M*R_M*np.cos(lat_rad)*np.sin(np.radians(azimuth))*1000 #  m/s
escape_vel_ideal = np.sqrt(2*G*M_M/((R_M + target_altitude)*1000)) - dV_earth_help  # m/s

dV_safety_factor = 1.1
dV = escape_vel_ideal*dV_safety_factor

print('Target Inclination: %.2f deg \nTarget Altitude: %.2f km \nCalculated Azimuth: %.2f deg\nTotal dV: %.2f m/s'%(target_inclination, target_altitude, azimuth, dV))

Target Inclination: 60.00 deg 
Target Altitude: 100.00 km 
Calculated Azimuth: 40.75 deg
Total dV: 11956.06 m/s


In [52]:
f1 = np.linspace(0, 1, 300)
f2 = 1 - f1

c_1 = g_0*Isp_1
c_2 = g_0*Isp_2

p2 = np.exp(f2*dV/c_2)*(1-fInert2)/(1-fInert2*np.exp(f2*dV/c_2))
p1 = np.exp(f1*dV/c_1)*(1-fInert1)/(1-fInert1*np.exp(f1*dV/c_1))

f_pay = p1*p2
pay_cons = f_pay[f_pay>0]
f1_cons = f1[f_pay>0]

lowest_ind = np.where(pay_cons == min(pay_cons))[0][0]
cor_f1 = f1_cons[lowest_ind]
cor_f2 = 1-cor_f1
cor_tot_mass = pay_cons[lowest_ind]

dV1 = cor_f1*dV
dV2 = cor_f2*dV

m_prop_2 = mpay*(1-np.exp(dV2/c_2))*(1-fInert2)/(1-fInert2*np.exp(dV2/c_2))
m_inert_2 = fInert2*m_prop_2/(1-fInert2)
m_final_2 = m_inert_2+mpay
m_initial_2 = m_final_2+m_prop_2

m_pay_1 = (mpay+m_initial_2)
m_prop_1 = m_pay_1*(1-np.exp(dV1/c_1))*(1-fInert1)/(1-fInert1*np.exp(dV1/c_1))
m_inert_1 = fInert1*m_prop_1/(1-fInert1)
m_final_1 = m_inert_2+m_pay_1
m_initial_1 = m_final_1+m_prop_1

tot_mass = m_initial_2 + m_initial_1

thrust_1 = twr_1*tot_mass*g_M
mdot_1 = thrust_1/c_1
t_b_1 = m_prop_1/mdot_1

thrust_2 = twr_2*m_final_1*g_M
mdot_2 = thrust_2/c_2
t_b_2 = m_prop_2/mdot_2

print('dV Splitting - 2 Stages')
print('dV Stage 1: %.2f m/s\ndV Stage 2: %.2f m/s'%(dV1, dV2))

if np.any(np.array([m_prop_2, m_inert_2, m_final_2, m_inert_1, m_final_1, m_initial_1]) < 0):
    print('Error: Mass Calculation Failed')
    
else:

    print('\nStage 1')
    print('Masses:')
    print('    Dry Mass:           %.2f kg'%(m_final_1))
    print('    Propellant Mass:    %.2f kg'%(m_prop_1))
    print('    Mass of Stage 2:    %.2f kg'%(m_pay_1))
    print('Engine Information:')
    print('    Thrust:             %.2f N'%(thrust_1))
    print('    Mass Flow Rate:     %.2f kg/s'%(mdot_1))
    print('    Burn Duration:      %.2f s'%(t_b_1))

    print('\nStage 2')
    print('Masses:')
    print('    Dry Mass:           %.2f kg'%(m_final_2))
    print('    Propellant Mass:    %.2f kg'%(m_prop_2))
    print('    Payload Mass:       %.2f kg'%(mpay))
    print('Engine Information:')
    print('    Thrust:             %.2f N'%(thrust_2))
    print('    Mass Flow Rate:     %.2f kg/s'%(mdot_2))
    print('    Burn Duration:      %.2f s'%(t_b_2))


dV Splitting - 2 Stages
dV Stage 1: 5478.19 m/s
dV Stage 2: 6477.86 m/s

Stage 1
Masses:
    Dry Mass:           120207.53 kg
    Propellant Mass:    433054.92 kg
    Mass of Stage 2:    100166.02 kg
Engine Information:
    Thrust:             8840131.69 N
    Mass Flow Rate:     2909.85 kg/s
    Burn Duration:      148.82 s

Stage 2
Masses:
    Dry Mass:           30041.51 kg
    Propellant Mass:    60124.52 kg
    Payload Mass:       10000.00 kg
Engine Information:
    Thrust:             1533576.50 N
    Mass Flow Rate:     460.26 kg/s
    Burn Duration:      130.63 s
